<a href="https://colab.research.google.com/github/ErikSeguinte/movie_data/blob/master/processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import cufflinks as cf
import numpy as np
from plotly import graph_objs as go
import altair as alt

In [ ]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [ ]:
cf.set_config_file(offline=True)

* I previously pulled CSV files from Kaggle, but the files were too big to host on github.
* I imported the files I wanted into pandas, and then exported them back out as compressed pickles.
* I was able to compress a 700MB csv to a 3 MB Pickle

In [2]:
try: 
    movies = pd.read_pickle('data/movies.pkl.xz')
    ratings = pd.read_pickle('data/ratings2.pkl.xz')
except:
    # Download pickles from github
    !wget https://github.com/ErikSeguinte/movie_data/raw/master/data/ratings2.pkl.xz
    !wget https://github.com/ErikSeguinte/movie_data/raw/master/data/movies.pkl.xz
    # Unpickle dataframes
    movies = pd.read_pickle('movies.pkl.xz')
    ratings = pd.read_pickle('ratings2.pkl.xz')

In [58]:
!wget 'https://datahub.io/core/cpi-us/r/0.csv'

--2020-03-01 21:10:54--  https://datahub.io/core/cpi-us/r/0.csv
Resolving datahub.io (datahub.io)... 104.24.112.103, 104.24.113.103, 2606:4700:3035::6818:7167, ...
Connecting to datahub.io (datahub.io)|104.24.112.103|:443... connected.
HTTP request sent, awaiting response...302 Found
Location: https://pkgstore.datahub.io/core/cpi-us/cpiai_csv/data/b17bfacbda3c08e51cd13fe544b8fca4/cpiai_csv.csv [following]
--2020-03-01 21:10:55--  https://pkgstore.datahub.io/core/cpi-us/cpiai_csv/data/b17bfacbda3c08e51cd13fe544b8fca4/cpiai_csv.csv
Resolving pkgstore.datahub.io (pkgstore.datahub.io)... 104.24.112.103, 104.24.113.103, 2606:4700:3036::6818:7067, ...
Connecting to pkgstore.datahub.io (pkgstore.datahub.io)|104.24.112.103|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 27030 (26K) [text/plain]
Saving to: ‘0.csv’

0.csv               100%[===================>]  26.40K  --.-KB/s    in 0.005s  

2020-03-01 21:10:55 (5.01 MB/s) - ‘0.csv’ saved [27030/27030]



In [ ]:
movies.head(1)

In [ ]:
ratings.shape

In [ ]:
movies.shape

## Clean Movie DF

In [ ]:
movies.dtypes

* Movies Dataframe has malformed data. `id` Should be numeric.
* After inspection, it looks like there are rows that are missing a comma somewhere, making columns not line up, and adding the wrong data to columns. Let's clean those up.
* All malformed rows have strings for IDs instead of numeric, so we will coerce them into numeric columns, and strings will be returned as `NaN`, which we'll then drop.

* `budget` and `revanue` should also be numeric, but Nans won't be dropped






In [3]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
movies = movies[movies['id'].notnull()]
movies = movies.set_index('id')

In [4]:
def to_numeric(df, labels):
    
    for label in labels:
        df[label] = pd.to_numeric(movies[label], errors='coerce').copy()
    return df

In [5]:
movies = to_numeric(movies, ['budget', 'revenue', 'vote_average'])

In [6]:
movies['release_date'] =pd.to_datetime(movies['release_date'], infer_datetime_format= True)

In [89]:
clean_movies = movies[['title', 'release_date','budget', 'revenue', 'runtime', 'vote_average', 'vote_count']]

## Process User Reviews
* User reviews come in a collection of individual reviews where a review gives a movie a score of 1 to 5.
* We will take the mean ratings for each movie

In [ ]:
ratings

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()
mean_rating = pd.DataFrame(scaler.fit_transform(ratings.groupby('movieId')[['rating']].mean()), columns = ["rating"])
#mean_rating = pd.DataFrame(ratings.groupby('movieId')[['rating']].mean(), columns = ["rating"])

In [10]:
# Aggregate mean ratings and number of votes per movie
movie_ratings =pd.DataFrame(ratings.groupby('movieId')[['rating']].agg(['mean', 'count']))['rating']
movie_ratings = movie_ratings.rename({'mean': 'rating', 'count': 'num_votes'}, axis = 1)

* Lets drop any movies with less than 10 votes. Those are more easily swayed by outliers and aren't reliable.

In [11]:
movie_ratings = movie_ratings[~(movie_ratings['num_votes'] < 100)]

* And now we merge the averaged ratings back with the movie database.
* Note that not all movies are present in the user votings.

In [12]:
movie_ratings = clean_movies.merge(movie_ratings, left_index = True, right_index=True)

In [34]:
movie_ratings[['title', 'rating']].nlargest(10, 'rating')

,title,rating
858.0,Sleepless in Seattle,4.339811
527.0,Once Were Warriors,4.266531
2019.0,Hard Target,4.255074
2959.0,License to Wed,4.230716
922.0,Dead Man,4.200820
1213.0,The Talented Mr. Ripley,4.178289
926.0,Galaxy Quest,4.174583
296.0,Terminator 3: Rise of the Machines,4.169975
1248.0,Hannibal Rising,4.157242
593.0,Solaris,4.152246


* The movie Database also provides a rating and suffer from a similar problem of some movies having a tiny sample size.

In [33]:
movie_ratings[['title', 'vote_average', 'vote_count']].sort_values(by='vote_average', ascending = False).nlargest(10, 'vote_average')

,title,vote_average,vote_count
238.0,The Godfather,8.5,6024.0
278.0,The Shawshank Redemption,8.5,8358.0
510.0,One Flew Over the Cuckoo's Nest,8.3,3001.0
550.0,Fight Club,8.3,9678.0
637.0,Life Is Beautiful,8.3,3643.0
424.0,Schindler's List,8.3,4436.0
155.0,The Dark Knight,8.3,12269.0
680.0,Pulp Fiction,8.3,8670.0
129.0,Spirited Away,8.3,3968.0
240.0,The Godfather: Part II,8.3,3418.0


In [57]:
movie_ratings[['title', 'revenue']].nlargest(10, 'revenue')

,title,revenue
597.0,Titanic,1.845034e+09
122.0,The Lord of the Rings: The Return of the King,1.118889e+09
58.0,Pirates of the Caribbean: Dead Man's Chest,1.065660e+09
1865.0,Pirates of the Caribbean: On Stranger Tides,1.045714e+09
155.0,The Dark Knight,1.004558e+09
671.0,Harry Potter and the Philosopher's Stone,9.764756e+08
12.0,Finding Nemo,9.403355e+08
767.0,Harry Potter and the Half-Blood Prince,9.339592e+08
121.0,The Lord of the Rings: The Two Towers,9.262874e+08
1893.0,Star Wars: Episode I - The Phantom Menace,9.243176e+08


## Inflation
* Inflation means that a 1940 dollar is worth more than a 2020 dollar. Let's adjust Revenue for that.
* The Consumer price index can be used to convert to standarized dollars.
* Here, we'll be using 2014 dollars.
* Years later than 2014 will not be adjusted.
$$ \textrm{adjusted dollars} = \frac{\textrm{New CPI}}{\textrm{Base CPI}}$$
* where x is the current cpi and y is the cpi of that year 

In [59]:
!wget 'https://datahub.io/core/cpi/r/cpi.csv'

--2020-03-01 21:20:54--  https://datahub.io/core/cpi/r/cpi.csv
Resolving datahub.io (datahub.io)... 104.24.112.103, 104.24.113.103, 2606:4700:3035::6818:7167, ...
Connecting to datahub.io (datahub.io)|104.24.112.103|:443...connected.
HTTP request sent, awaiting response...302 Found
Location: https://pkgstore.datahub.io/core/cpi/cpi_csv/data/04cb8fe18892497287d23e20d0e1ceb9/cpi_csv.csv [following]
--2020-03-01 21:20:55--  https://pkgstore.datahub.io/core/cpi/cpi_csv/data/04cb8fe18892497287d23e20d0e1ceb9/cpi_csv.csv
Resolving pkgstore.datahub.io (pkgstore.datahub.io)...104.24.113.103, 104.24.112.103, 2606:4700:3035::6818:7167, ...
Connecting to pkgstore.datahub.io (pkgstore.datahub.io)|104.24.113.103|:443... connected.
HTTP request sent, awaiting response...200 OK
Length: 254135 (248K) [text/plain]
Saving to: ‘cpi.csv’

cpi.csv             100%[===================>] 248.18K  --.-KB/s    in 0.09s   

2020-03-01 21:20:56 (2.61 MB/s) - ‘cpi.csv’ saved [254135/254135]



In [62]:
cpi = pd.read_csv('cpi.csv')
cpi

,Country Name,Country Code,Year,CPI
0,Afghanistan,AFG,2004,63.131893
1,Afghanistan,AFG,2005,71.140974
2,Afghanistan,AFG,2006,76.302178
3,Afghanistan,AFG,2007,82.774807
4,Afghanistan,AFG,2008,108.066600
...,...,...,...,...
6931,Zambia,ZMB,2010,100.000000
6932,Zambia,ZMB,2011,106.429397
6933,Zambia,ZMB,2012,113.428087
6934,Zambia,ZMB,2013,121.342732


In [67]:
cpi = cpi[cpi['Country Name'] == 'United States'][['Year', 'CPI']]

In [69]:
cpi = cpi.set_index(cpi['Year'])

In [76]:
cpi.loc[1994,'CPI']

67.9758134970225

In [132]:
def adjust_dollars(value, year):
    year = int(year)
    try:
        current = cpi.loc[2014,'CPI']
        base = cpi.loc[year,'CPI']
        adjusted_value = value * (current/base)
        return adjusted_value
    except: 
        return value

In [90]:
clean_movies['year']= clean_movies['release_date'].dt.year

In [129]:
df = clean_movies[clean_movies['revenue'].notnull() & clean_movies['year'].notnull()]
df

,title,release_date,budget,revenue,runtime,vote_average,vote_count,year
id,,,,,,,,
862.0,Toy Story,1995-10-30,30000000.0,373554033.0,81.0,7.7,5415.0,1995.0
8844.0,Jumanji,1995-12-15,65000000.0,262797249.0,104.0,6.9,2413.0,1995.0
31357.0,Waiting to Exhale,1995-12-22,16000000.0,81452156.0,127.0,6.1,34.0,1995.0
11862.0,Father of the Bride Part II,1995-02-10,NaN,76578911.0,106.0,5.7,173.0,1995.0
949.0,Heat,1995-12-15,60000000.0,187436818.0,170.0,7.7,1886.0,1995.0
...,...,...,...,...,...,...,...,...
280422.0,All at Once,2014-06-05,750000.0,3.0,NaN,6.0,4.0,2014.0
240789.0,The Miracle,2009-10-09,NaN,50656.0,110.0,6.3,3.0,2009.0
62757.0,Savages,2006-11-23,800000.0,1328612.0,100.0,5.8,6.0,2006.0


In [138]:
adjusted = pd.DataFrame([adjust_dollars(x,y) for x,y in zip(df['revenue'], df['year'])], index = df.index, columns = ['adjusted_revenue'])

In [139]:
clean_movies.merge(adjusted, left_index=True, right_index=True).nlargest(10,'adjusted_revenue')

,title,release_date,budget,revenue,runtime,vote_average,vote_count,year,adjusted_revenue
id,,,,,,,,,
19995.0,Avatar,2009-12-10,237000000.0,2.787965e+09,162.0,7.2,12114.0,2009.0,3.076449e+09
11.0,Star Wars,1977-05-25,11000000.0,7.753980e+08,121.0,8.1,6778.0,1977.0,3.028728e+09
597.0,Titanic,1997-11-18,200000000.0,1.845034e+09,194.0,7.5,7770.0,1997.0,2.721128e+09
9552.0,The Exorcist,1973-12-26,8000000.0,4.413061e+08,122.0,7.5,2046.0,1973.0,2.351851e+09
15121.0,The Sound of Music,1965-03-02,8200000.0,2.862143e+08,174.0,7.4,966.0,1965.0,2.148826e+09
578.0,Jaws,1975-06-18,7000000.0,4.706540e+08,124.0,7.5,2628.0,1975.0,2.069945e+09
140607.0,Star Wars: The Force Awakens,2015-12-15,245000000.0,2.068224e+09,136.0,7.5,7993.0,2015.0,2.068224e+09
601.0,E.T. the Extra-Terrestrial,1982-04-03,10500000.0,7.929653e+08,115.0,7.3,3359.0,1982.0,1.945322e+09
12230.0,One Hundred and One Dalmatians,1961-01-25,4000000.0,2.158800e+08,79.0,6.8,1643.0,1961.0,1.708505e+09


In [15]:
# Dropping low sample size averages from Votes on the movie database
movie_ratings = movie_ratings[~(movie_ratings['vote_count'] < 100)]

In [ ]:
movie_ratings.dtypes

In [16]:
movie_ratings.corr()

,budget,revenue,runtime,vote_average,vote_count,rating,num_votes
budget,1.000000,0.647264,0.236568,-0.290705,0.394566,-0.004189,0.053151
revenue,0.647264,1.000000,0.236660,0.007136,0.680686,-0.008843,0.064237
runtime,0.236568,0.236660,1.000000,0.243459,0.251967,0.001315,0.022148
vote_average,-0.290705,0.007136,0.243459,1.000000,0.294917,0.046826,0.106657
vote_count,0.394566,0.680686,0.251967,0.294917,1.000000,-0.014337,0.057152
rating,-0.004189,-0.008843,0.001315,0.046826,-0.014337,1.000000,0.268050
num_votes,0.053151,0.064237,0.022148,0.106657,0.057152,0.268050,1.000000


In [17]:
# Adding a year and decade to examine trends over time
movie_ratings['year'] = movie_ratings['release_date'].dt.year

In [18]:
movie_ratings['decade'] = [x - (x%10) for x in movie_ratings['year']]

In [ ]:
#enable_plotly_in_cell()
movie_ratings.groupby('year')['vote_average'].mean().iplot(kind='bar')

In [36]:
alt.Chart(movie_ratings).mark_bar().encode(
    alt.Y('mean(vote_average)'),
    alt.X('year')
)

alt.Chart(...)

In [37]:
alt.Chart(movie_ratings).mark_bar().encode(
    alt.Y('mean(rating)'),
    alt.X('year')
)

alt.Chart(...)

In [ ]:
movie_ratings

In [ ]:
# enable_plotly_in_cell()
movie_ratings.groupby('decade')['rating'].mean().iplot(kind='bar')

* I'd like to compare the votes from TMB to the user ratings, but they are on different scales. We'll use standard scaler to normalize them so we can more easily compare.

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
to_scale = movie_ratings[['vote_average', 'rating']].notnull()
scaled = scaler.fit_transform(to_scale)
movie_ratings = movie_ratings.merge(
    pd.DataFrame(
        scaled,
        index = movie_ratings.index,
        columns = ['scaled_tmdb_vote', 'scaled_user_rating']
    ),
    left_index = True,
    right_index = True,
)

In [28]:
grouped_ratings = pd.melt(movie_ratings, id_vars = 'year', value_vars=['scaled_tmdb_vote', 'scaled_user_rating'])
grouped_ratings

,year,variable,value
0,1995.0,scaled_tmdb_vote,-0.338399
1,1977.0,scaled_tmdb_vote,1.629186
2,2003.0,scaled_tmdb_vote,1.014316
3,1994.0,scaled_tmdb_vote,1.752160
4,1999.0,scaled_tmdb_vote,1.383238
...,...,...,...
2589,1966.0,scaled_user_rating,-0.119128
2590,1969.0,scaled_user_rating,0.445003
2591,1982.0,scaled_user_rating,0.765828
2592,1955.0,scaled_user_rating,0.355415


In [30]:
alt.Chart(grouped_ratings).mark_bar().encode(
    alt.Y('mean(value)'),
    alt.X('variable'),
    column='year'
    
)

alt.Chart(...)

In [ ]:
movie_ratings.groupby('year')[['scaled_user_rating', 'scaled_tmdb_vote']].mean().iplot(kind='bar')

In [ ]:
movie_ratings.groupby('year')['vote_average'].mean()

In [ ]:
movie_ratings.groupby('year')['scaled_tmdb_vote'].mean().index

In [32]:
traces = [
    go.Bar(name='TMDB rating',
        x = movie_ratings.groupby('year')['scaled_tmdb_vote'].mean().index,
        y = movie_ratings.groupby('year')['scaled_tmdb_vote'].mean()
    ),
        go.Bar(name='user rating',
        x = movie_ratings.groupby('year')['scaled_user_rating'].mean().index,
        y = movie_ratings.groupby('year')['scaled_user_rating'].mean()
    )
]

go.Figure(data = traces,
    layout_xaxis_tick0 = 1890
)

In [55]:
from sklearn.decomposition import PCA

pca = PCA(1)

pca_df = pd.DataFrame(pca.fit_transform(scaled), index=to_scale.index, columns = ['PCA'])

movie_ratings = movie_ratings.merge(pca_df, left_index = True, right_index = True)
movie_ratings.head(2)

,title,release_date,budget,revenue,runtime,vote_average,vote_count,rating,num_votes,year,decade,scaled_tmdb_vote_x,scaled_user_rating_x,scaled_tmdb_vote_y,scaled_user_rating_y,scaled_tmdb_vote,scaled_user_rating,PCA_x,PCA_y,PCA
5.0,Four Rooms,1995-12-09,4000000.0,4300000.0,98.0,6.5,539.0,3.079565,15258,1995.0,1990.0,-0.338399,-0.425327,-0.338313,-0.426370,0.17291,0.0,-0.17291,-0.17291,-0.17291
11.0,Star Wars,1977-05-25,11000000.0,775398007.0,121.0,8.1,6778.0,3.660591,19475,1977.0,1970.0,1.629186,0.732299,1.630764,0.731907,0.17291,0.0,-0.17291,-0.17291,-0.17291


In [56]:
movie_ratings[['title', 'PCA']].nlargest(10, 'PCA')

,title,PCA
1488.0,Mongolian Ping Pong,5.78337
2033.0,Infinity,5.78337
2397.0,Fallout,5.78337
3024.0,The Strange Case of Dr. Jekyll and Mr. Hyde,5.78337
3118.0,Frankenstein 90,5.78337
3576.0,Calling Dr. Gillespie,5.78337
3966.0,Blondie Knows Best,5.78337
4419.0,Les Misérables,5.78337
4709.0,Holiday for Henrietta,5.78337
4979.0,Windows on Monday,5.78337


In [38]:
# enable_plotly_in_cell()
trace = go.Box(
    x = movie_ratings[movie_ratings['decade'].notnull()]['decade'],
    y = movie_ratings[movie_ratings['decade'].notnull()]['rating'],
    
)
go.Figure(
    trace,
    layout_xaxis_title = "Decade",
    layout_yaxis_title = "Movie Rating",
    layout_title = "Movie Ratings by decade"
)


In [ ]:
movie_ratings['q_budget'] = pd.qcut(movie_ratings['budget'], labels = ['vlow', 'low', 'med', 'high', 'vhigh'], q = 5)

In [ ]:
budget_ratings = movie_ratings[['title', 'budget', 'revenue', 'rating']].dropna()

In [ ]:
budget_ratings.corr()

In [ ]:
trace = go.Scatter(
    y = budget_ratings['rating'],
    x = budget_ratings['revenue'],
    mode = 'markers'
)

go.Figure(
    trace,
    layout_xaxis_title = "Budget",
    layout_yaxis_title = "Movie Rating",
    layout_title = "Movie Ratings by budget",
    
)

In [ ]:
movie_ratings.shape

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA

In [ ]:
all_ratings = movie_ratings[['vote_average','rating']].dropna()
all_ratings

In [ ]:
all_ratings.isnull().sum()

In [ ]:
scaler = StandardScaler()
x =scaler.fit_transform(all_ratings)

In [ ]:
pca = PCA(1)

In [ ]:
x = pca.fit_transform(x)

In [ ]:
scaled_ratings = pd.DataFrame(x, index = all_ratings.index, columns=['scaled_rating'])

In [ ]:
scaled_ratings

In [ ]:
movie_ratings = movie_ratings.merge(scaled_ratings, left_index=True, right_index=True)

In [ ]:
movie_ratings.corr()

In [ ]:
enable_plotly_in_cell()
movie_ratings.groupby('year')['scaled_rating'].mean().iplot(kind='bar')

In [ ]:
movie_ratings.groupby('year')['scaled_rating'].mean().iplot(kind='bar', title = "Scaled Rating by Year", xTitle="year", yTitle="Scaled Rating")

In [ ]:
movie_ratings.groupby('decade')['scaled_rating'].mean().iplot(kind='bar')

In [ ]:
budget_ratings = movie_ratings[['budget', 'rating']].dropna()

In [ ]:
budget_ratings['q_budget'] = pd.qcut(budget_ratings['budget'], q = 5, labels = ['vlow', 'low', 'med', 'high', 'blockbuster'])


In [ ]:
enable_plotly_in_cell()
budget_ratings.groupby('q_budget')['rating'].mean().iplot(kind='bar')

In [ ]:
movie_ratings.nlargest(25, 'rating')

In [ ]:
enable_plotly_in_cell()
trace = go.Scatter(
    x = movie_ratings["budget"],
    y = movie_ratings['revenue'],
    mode = "markers"
)

go.Figure(trace)